In [1]:
%reload_ext autoreload
%autoreload 2

In [8]:
from sodapy import Socrata
from plotly import graph_objects as go
from datetime import datetime, date, time, timedelta
from ipywidgets import widgets, interact, Layout, Button, HBox, VBox
import pandas as pd
import math
import numpy as np
import utils
import plotly

pd.options.display.max_colwidth = 200
import warnings
warnings.filterwarnings('ignore')

print('import done')

import done


In [5]:
### Get Data from Socrata


## Build client
usrname = "jjensen@mointern.nyc.gov"
pword = "Timspw19!"
client = Socrata("data.cityofnewyork.us","TnnbmTZvKbFz7tJ4NpzkfUG6C", username=usrname, password=pword)

## Make API calls
# auth_json_downloads = client.get("beb8-u7wp", limit=2000000)
# auth_json_dailyviews = client.get("qiif-mxcx", limit=50000)
auth_json_authentications = client.get("8aih-pmax", limit = 20000)
auth_json_datasetfacts = client.get("gzid-z3nh", limit = 50000)
auth_json_public_inventory = client.get("5tqd-u88y", limit=5000)
auth_json_api_calls = client.get("u7pi-wq4y", limit=2000)
auth_json_the_plan = client.get("qj2z-ibhs", limit=5000)


## Get the private asset inventory
## only necessary because the columns `asset_type` is only in the private and not the public asset inventory
auth_json_asset_inventory = client.get("r8cp-r4rc", limit = 40000)

In [ ]:
### Generate base dataframes to build off of throughout the notebook

# dv = pd.DataFrame(auth_json_dailyviews)
# dauth = pd.DataFrame(auth_json_authentications)
dfacts = pd.DataFrame(auth_json_datasetfacts)
# ddownloads = pd.DataFrame(auth_json_downloads)
public = pd.DataFrame(auth_json_public_inventory)
private = pd.DataFrame(auth_json_asset_inventory)
the_plan = pd.DataFrame(auth_json_the_plan)

In [ ]:
### Data Cleaning

# Create merged_filter, the dataframe that has only assets defined as datasets
dataset_filter_list = ['dataset','filter', 'gis map']
## Cleaning dates entered in the wrong format
bad_index_list = list(public.query('date_made_public == "August 9, 2-019"').index)
for val in bad_index_list:
    public.at[val, 'date_made_public'] = '8/9/2019'

# need to merge in private so that we can get the asset type
dupes = private[['u_id', 'agency', 'type','name', 'parent_uid','derived_view']]

public = utils.date_transformation(public, 'date_made_public')
# left join public and private assets to get necessary columns from private asset inventory, such as type
merged_total = pd.merge(public, dupes, how='left', on='u_id')
# merged_filter is the merged_total dataframe but filtered to only include assets considered as datasets
merged_filter = merged_total[merged_total.type.isin(dataset_filter_list)]
merged_filter.agency_x = merged_filter.agency_x.astype(str)


## Derived vs Parent Assets

### What is a dataset? 
A dataset is defined as an asset where derived_view is False and asset_type is either gis map, dataset, or filter.

### What is an asset? 
An asset is the term used to describe every object on the NYC Open Data Portal. Assets can refer to either parent datasets, objects derived from parent datasets, or objects that are not derived and not considered datasets -- such as, files or href links. 

In [ ]:
# Calculations of total assets, total datasets, nonderived and derived assets below. 

not_derived = merged_total.query('derived_view == "false"')
dataset_df = not_derived[not_derived.type.isin(dataset_filter_list)][['name_x','agency_x','type','u_id','date_made_public']]
num_assets = merged_total.shape[0]
num_datasets = merged_filter.groupby('derived_view').size()['false']
num_not_derived_assets = merged_total.query('derived_view == "false"')[~merged_total.type.isin(dataset_filter_list)].shape[0]
num_files = merged_total.query('derived_view == "false"')[~merged_total.type.isin(dataset_filter_list)].groupby('type').size()['file']
num_links = merged_total.query('derived_view == "false"')[~merged_total.type.isin(dataset_filter_list)].groupby('type').size()['href']
num_derived_assets = merged_total.query('derived_view == "true"').shape[0]

print('There are {} assets on the NYC Open Data Portal.'.format(num_assets))
print('There are {} datasets on the NYC Open Data Portal.'.format(num_datasets))
print('There are {} assets that are not derived on the NYC Open Data Portal -- {} files and {} links.'\
      .format(num_not_derived_assets, num_files, num_links))
print('There are {} derived assets on the NYC Open Data Portal.'.format(num_derived_assets))


## Number of Datasets on NYC Open Data Portal

In [ ]:
dataset_df.sort_values(by='date_made_public', ascending=False,inplace=True)
null_public = dataset_df[dataset_df.date_made_public.isnull()].shape[0]

print('There are {} datasets without a value for date_made_public.'.format(null_public))

dataset_df['date_made_public'] = pd.to_datetime(dataset_df['date_made_public'])
dataset_df = dataset_df[pd.notnull(dataset_df['date_made_public'])]
dataset_df.reset_index(inplace=True, drop=True)

print('----------Dropping {} rows that do not have a timestamp for date_made_public----------'.\
    format(null_public))

In [ ]:
fy_dict, fy_filter_list = utils.build_fy_dictionary(dataset_df, 'date_made_public')

# calls the create_fy_columns function to create binary columns that allow us to tally datasets by FY they were created
published_datasets_by_fy = utils.create_fy_columns(dataset_df, fy_dict, 'date_made_public')
published_assets_by_fy = utils.create_fy_columns(merged_total, fy_dict, 'date_made_public')

In [ ]:
# Produces dataframe with the number of datasets per FY by filtering out assets not defined in fy_filter_list
num_datasets = pd.DataFrame(published_datasets_by_fy[fy_filter_list].sum())\
    .reset_index().rename(columns={'index':'FY', 0:'Number of Datasets on ODP'})

# Produces dataframe with the number of datasets per FY by filtering out assets not defined in fy_filter_list
num_assets = pd.DataFrame(published_assets_by_fy[fy_filter_list].sum())\
    .reset_index().rename(columns={'index':'FY', 0:'Number of Assets on ODP'})

assets_by_type = pd.DataFrame(merged_total.groupby('type').size())\
    .reset_index().sort_values(ascending=False, by=0)\
    .rename(columns={'type':'asset_type', 0:'Count'})

In [ ]:

# plotly chart displaying number of datasets at the end of each fiscal year
fig = go.Figure(data=[go.Bar(
    x=num_datasets['FY'],
    y=num_datasets['Number of Datasets on ODP'],
    text=num_datasets['Number of Datasets on ODP'],
    textposition='auto')])

fig.update_layout(
     title=go.layout.Title(
        text="Total Datasets at the End of the Fiscal Year",
        xref="paper",
        x=.5)
)
fig.show()

In [ ]:
# Number of Assets per FY

fig = go.Figure(data=[go.Bar(
    x=num_assets['FY'],
    y=num_assets['Number of Assets on ODP'],
    text=num_assets['Number of Assets on ODP'],
    textposition='auto')])
fig.update_layout(
     title=go.layout.Title(
        text="Number of Assets at the End of the Fiscal Year",
        xref="paper",
        x=.5)
    )
fig.show()

,agency_x,name_x,description,dataset_link,u_id,dataset_from_the_open_data_plan,original_scheduled_publication_date,most_recent_scheduled_publication_date,date_made_public,has_data_dictionary,...,FY_2016,End_of_FY_2016,FY_2017,End_of_FY_2017,FY_2018,End_of_FY_2018,FY_2019,End_of_FY_2019,FY_2020,As of Today
0,311,311 Service Requests for 2004,311 Service Requests for 2004. This is historical data and will not be updated.,{'url': 'https://data.cityofnewyork.us/d/sqcr-6mww'},sqcr-6mww,No,N/A,N/A,2011-10-20,Yes,...,0,1,0,1,0,1,0,1,0,1
1,311,311 Service Requests for 2006,311 Service Requests for 2006. This is historical data and will not be updated.,{'url': 'https://data.cityofnewyork.us/d/hy4q-igkk'},hy4q-igkk,No,N/A,N/A,2011-10-20,Yes,...,0,1,0,1,0,1,0,1,0,1
2,311,311 Call Center Inquiry,"<div>The 311 Call Center Inquiry dataset contains information on all agent-handled calls to the City’s 311 information line, including date, time and topic.<br></div><div><a target=""_blank"" rel=""n...",{'url': 'https://data.cityofnewyork.us/view/tdd6-3ysr'},tdd6-3ysr,No,N/A,N/A,2016-05-13,Yes,...,1,1,0,1,0,1,0,1,0,1
3,311,311 Service Requests for 2007,311 Service Requests for 2007. This is historical data and will not be updated.,{'url': 'https://data.cityofnewyork.us/d/aiww-p3af'},aiww-p3af,No,N/A,N/A,2011-10-20,Yes,...,0,1,0,1,0,1,0,1,0,1
4,311,311 Call Center Inquiry,"The 311 Call Center Inquiry dataset contains information on all agent-handled calls to the City’s 311 information line, including date, time and topic.\r\n\r\nClick <a href=""https://data.cityofnew...",{'url': 'https://data.cityofnewyork.us/d/wewp-mm3p'},wewp-mm3p,No,N/A,N/A,2016-06-13,Yes,...,1,1,0,1,0,1,0,1,0,1


In [ ]:
# Calling attention to Total Assets by Asset Type for two reasons: (1) internally we use the term "dataset",
# however, there is also an asset type called "dataset." When we refer to a dataset on the Open Data Portal, 
# we're referring to an asset of that is not derived and is of type 'dataset','gis map', or 'filter';
# (2) We have a significant number of assets that live on the portal but are not datasets. When speaking of datasets,
# these assets (href, file, datalens) should be excluded. This chart displays all assets, derived or not derived,
# by their asset type. 

fig = go.Figure(data=[go.Bar(
    x=assets_by_type['asset_type'],
    y=assets_by_type['Count'],
    text=assets_by_type['Count'],
    textposition='auto')])

fig.update_layout(
     title=go.layout.Title(
        text="Total Assets by Asset Type",
        xref="paper",
        x=.5)
    )
fig.show()

### Select an asset type from the following dropdown menu to view all corresponding assets on the NYC Open Data Portal.

In [ ]:

output_type = widgets.Output()
dropdown_type = widgets.Dropdown(options = utils.unique_sorted_values_plus_ALL(merged_total.type),\
                                layout=Layout(width='20%'))
dropdown_type.layout.margin = "0px 0px 0px 400px"

def display_assets(asset_type):
    output_type.clear_output()
    df = merged_total[['name_x','agency_x','u_id','type']]
    df.rename(columns={'agency_x':'agency','name_x':'asset_name','type':'asset_type'}, inplace=True)
    with output_type:
        with pd.option_context("display.max_rows", 10000):
            if (asset_type == utils.ALL):
                display(df)
            else:
                display(df[df.asset_type == asset_type])

def dropdown_type_eventhandler(change):
    display_assets(change.new)

dropdown_type.observe(dropdown_type_eventhandler, names='value')

display(dropdown_type)
display(output_type)


In [132]:
# Number of Unique Agencies that Published Dataset per Fiscal Year

published_datasets_by_fy.groupby(['fy_16','fy_17','fy_18','fy_19'], as_index=True)\
    ['agency_x'].nunique().reset_index()

KeyError: 'fy_16'

# Automations

This chart displays the total number of automated datasts on the Open Data Portal for a given month. Since we do not
have a timestamp for when a dataset was automated, the count of automated datasets is calculated using the timestamp for when a dataset was made public. The total count for the current month reflects the number of automated datasets live on the Open Data Portal. The caveat here is that if an older dataset was automated it would also be reflected in the month for when that dataset was made public. To improve this chart (and our understand of automation) we should add a timestamp for when a dataset was automated to the asset inventory.

In [ ]:
cumulative_automations = pd.DataFrame(merged_filter.query('automation == "Yes" & date_made_public')\
    .groupby(pd.Grouper(key='date_made_public', freq=('M'))).size().cumsum()).rename(columns={0:'Count'}).last('18M')

fig = go.Figure(data=[go.Bar(
    x=cumulative_automations.index,
    y=cumulative_automations['Count'],
    text=cumulative_automations['Count'],
    textposition='auto')])

fig.update_layout(
     title=go.layout.Title(
        text="Cumulative Total Automated Datasets in the Last 18 Months",
        xref="paper",
        x=.5)
     )
fig.show()

In [ ]:
public['last_update_date_data'] = pd.to_datetime(public['last_update_date_data'])
merged_filter['last_update_date_data'] = pd.to_datetime(public['last_update_date_data'])
merged_filter['last_update_date_data'] = merged_filter['last_update_date_data'].dt.tz_convert(None)

In [ ]:
# merged filter is a copy of the asset inventory that is filtered to only include assets defined as datasets
freshness_df = merged_filter[['name_x','agency_x','u_id','update_frequency',\
                              'date_made_public','last_update_date_data','automation']]

# Remove datasets with update frequencies for which we cannot determine freshness
freshness_df = freshness_df[~freshness_df['update_frequency'].isin(['Historical Data', 'As needed'])].reset_index(drop=True)
freshness_df.rename(columns={'name_x':'dataset_name','agency_x':'agency','u_id':'asset_id','last_update_date_data':'date_last_updated',\
                            'automation':'automated'}, inplace=True)

# Removes automated datasets 
# freshness_df = freshness_df[freshness_df['automation'] != "Yes"]

In [ ]:
FREQ_DICTIONARY = {
    
    'Daily' : [np.timedelta64(1, 'D')],
    'Weekly' : [np.timedelta64(1, 'W')],
    'Biweekly ': [np.timedelta64(4, 'D')],
    'Monthly': [np.timedelta64(1,'M')],
    'Quarterly':[np.timedelta64(3, 'M')],
    '2 to 4 times per year' : [np.timedelta64(6, 'M')],
    'Triannually': [np.timedelta64(4, 'M')],
    'Biannually ': [np.timedelta64(6, 'M')],
    'Annually': [np.timedelta64(1, 'Y')],
    'Weekdays': [np.timedelta64(2, 'D')],
    'Hourly': [np.timedelta64(1, 'h')],
    'Several times per day': [np.timedelta64(1, 'D')]
}

def calculate_freshness(df, date_col, fresh_col):
    '''
    Returns the asset inventory dataframe with an additional true/false column that
    indicates whether or not a dataset is fresh. Fresh is defined as having a 
    last_update_date_data timestamp within the stated update frequency
    
    Inputs:
        df: a dataframe, one that has filtered inapplicable update frequencies
            and all automated datasets
        date_col: last updated timestamp
        fresh_col: name of the binary fresh or stale column
    Returns:
        dataframe indicating if each dataset is stale or fresh
    '''
    
    df[fresh_col] = np.nan
    
    for frequency in df.update_frequency.unique():
                    
        
        temp = df.query('update_frequency == "{}"'.format(frequency))  
        today = datetime.today()
        idx = temp.index
        
#         if frequency == 'Weekdays':
#             np.busday_count(today, temp['last_update_date_data'])
            
        df.loc[idx, fresh_col] = (today - temp[date_col]) < FREQ_DICTIONARY[frequency][0]
    
    return df

## Measuring Data Freshness 

In [ ]:
# Percent of datasets fresh or stale
fresh_df = calculate_freshness(freshness_df, 'date_last_updated', 'fresh')
fresh_df['agency'] = fresh_df['agency'].astype(str)

# get index value for all automated datasets
auto_idx = np.where(freshness_df['automated']=='Yes')

# update fresh_df to be true for all automated datasets
fresh_df.fresh.iloc[auto_idx] = True

# sort fresh dataframe
fresh_df.sort_values(by=['fresh','date_last_updated'], inplace=True)


pct_fresh = fresh_df.groupby('fresh').size().reset_index().rename(columns={0:'count'})
pct_fresh['pct'] = pct_fresh['count'].apply(lambda x: x/pct_fresh['count'].sum())


# Stale datasets by update_frequency
uf = fresh_df.query('fresh == False').groupby('update_frequency')\
    .size().reset_index().rename(columns={0:'Count'}).sort_values(by='Count')

# 324 are annual, which could mean they're mistagged and should be historical
    
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']


labels = uf['update_frequency']
values = uf['Count']

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_layout(
     title=go.layout.Title(
        text="Stale Datasets According to their Update Frequency",
        xref="paper",
        x=.5)
     )
           
fig.show()

In [ ]:
stacked = fresh_df.groupby('agency')['fresh'].value_counts()\
    .unstack().fillna(0).reset_index().rename(columns={False:'Stale',True:'Fresh'})\
    .sort_values(by='Fresh',ascending=False)

fig = go.Figure(data=[
    go.Bar(name='Stale', x=stacked['agency'], y=stacked['Stale']),
    go.Bar(name='Fresh', x=stacked['agency'], y=stacked['Fresh'])
])

fig.update_layout(
     title=go.layout.Title(
        text="How Stale Is Each Agency's Data?",
        xref="paper",
        x=.5)
     )

fig.update_xaxes(showticklabels=False)
# Change the bar mode
fig.update_layout(barmode='stack')

fig.update_layout(
    autosize=False,
    width=1000,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),

)
fig.show()

## Data Freshness by Agency

Selecting an Agency from the dropdown menu below will present a pie chart displaying the percentage of datasets that are within their update frequency (fresh) and the percentage that have not received new data within their update frequency (stale). It will also present a table with the agency's datasets as well as a column that indicates whether the dataset is stale or not. 

In [ ]:
# The following code uses ipywidgets to generate the interact dropdown menu and the corresponding pie chart and table based
# on the user's selection. 

output = widgets.Output()
plot_output = widgets.Output()
output_agency = widgets.Output()

dropdown_agency = widgets.Dropdown(options = utils.unique_sorted_values_plus_ALL(fresh_df.agency))


def common_filtering(agency):
    output.clear_output()
    output_agency.clear_output()
    plot_output.clear_output()
    
    if (agency == utils.ALL):
        common_filter = fresh_df
    else:
        common_filter = fresh_df[(fresh_df.agency == agency)]
        
    with plot_output:

        df = common_filter.fresh.value_counts().reset_index().rename(columns={'index':'status','fresh':'count'})

        colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

        labels = df['status']
        values= df['count']

        fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

        fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                          marker=dict(colors=colors, line=dict(color='#000000', width=2)))
        fig.update_layout(
            title=go.layout.Title(
            text="The Datasets Are Up-to-Date",
            xref="paper",
            x=.5)
        )
        fig.show()
        
    with output_agency:
        with pd.option_context("display.max_rows", 10000):
            if (agency == utils.ALL):
                display(fresh_df)
            else:
                display(fresh_df[fresh_df.agency == agency])

def dropdown_agency_eventhandler(change):
    common_filtering(change.new)

dropdown_agency.observe(dropdown_agency_eventhandler, names='value')

display(dropdown_agency)

display(plot_output)

display(output_agency)

In [ ]:
## data cleanup
make_int = ['asset_columns', 'asset_rows']

for col in make_int:
    dfacts[col] = pd.to_numeric(dfacts[col])

dfacts['date'] = pd.to_datetime(dfacts['date'])

tables = dfacts[dfacts.derived_asset_type.isin(["table"])]
na = dfacts.query('derived_asset_type == "n/a"')
grouped = dfacts.groupby('asset_title')

dnum_rows = {}

for name, group in grouped:
    temp = group[group['date'] == group['date'].max()].reset_index()
    num_rows = temp.asset_rows[0]
    agency = temp['agency'][0]
    
    dnum_rows[agency] = dnum_rows.get(agency, 0) + num_rows

In [ ]:
# generates the dataframes necessary for the dataset vs rows scatter plot
clean_count = merged_filter.groupby('agency_x').size().sort_values(ascending=True)\
    .reset_index().rename(columns={'agency_x':'agency', 0:'count'})

df = pd.DataFrame.from_dict(dnum_rows, orient='index').rename_axis('agency').reset_index().rename(columns={0:'count'})

clean_count.columns = ['agency', 'numdatasets']
df.columns = ['agency', 'numrows']
rv = pd.merge(df, clean_count, on='agency')

rv['lognumrows'] = rv['numrows'].apply(lambda x: math.log(x))

## Publishing Data

Select an agency from the following dropdown menu to see how many datasets that agency has published in each year. 

In [ ]:
# Generages dataframes necessary for interactive bar chart to see number of datasets published per agency per year. 
merged_filter = merged_filter[pd.notnull(merged_filter['date_made_public'])]
merged_filter['year_made_public'] = merged_filter.date_made_public.dt.year.astype(int)

yearly_totals_per_agency = merged_filter.groupby(['agency_x','year_made_public']).size().unstack().fillna(0)


In [ ]:
# code to visualize agency publishing and generate interactive dropdown 

pub_output = widgets.Output()
bar_plot_output = widgets.Output()

dropdown_agency_name = widgets.Dropdown(options = utils.unique_sorted_values_plus_ALL(
                                                        yearly_totals_per_agency.index, all=False))


def plot_agency_total(agency):
    pub_output.clear_output()
    bar_plot_output.clear_output()
    
    common_filter = yearly_totals_per_agency[(yearly_totals_per_agency.index == agency)]
    cf = common_filter.loc[agency].reset_index().rename(columns={'year_made_public':'Year',\
                                                                agency:'Number of Datasets Published'})
    
    with bar_plot_output:

        fig = go.Figure(data=[go.Bar(
            x=cf['Year'],
            y=cf['Number of Datasets Published'],
            text=cf['Number of Datasets Published'],
            textposition='auto')])

        fig.update_layout(
             title=go.layout.Title(
                text="Total Datasets Published by the {}".format(agency),
                xref="paper",
                x=.5)
        )
        fig.show()

def dropdown_agency_name_eventhandler(change):
    plot_agency_total(change.new)
        
dropdown_agency_name.observe(dropdown_agency_name_eventhandler, names='value')
        
display(dropdown_agency_name)

display(bar_plot_output)

In [ ]:
fig = go.Figure(data=go.Scatter(x=rv['numdatasets'],
                                y=rv['lognumrows'],
                                mode='markers',
#                                 marker=dict(
#                                     color=rv['numdatasets'], #set color equal to a variable
#                                     colorscale='Jet', # one of plotly colorscales
#                                     showscale=False
#                                 ),
#                                 marker_color=rv['lognumrows'],
                                text=rv['agency'])) # hover text goes here

fig.update_layout(
     title=go.layout.Title(
        text="Density of Agency Data: Total Number of Rows Compared to Number of Datasets",
        xref="paper",
        x=.5),
    
     xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Total Number of Datasets",
            font=dict(
#                 family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Log Number of Total Rows",
            font=dict(
#                 family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)
fig.update_layout(
    autosize=False,
    width=1000,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
#     paper_bgcolor="LightSteelBlue",
)

fig.show()